# Notebook zu Gleitpunktzahlen im IEEE-Standard (Double-Precision)

---

# Arbeiten mit einem jupyter notebook

- Mit `help(Ausdruck)` können Sie sich Informationen zu "Ausdruck" anzeigen lassen.
- Sie können in den Zellen, die mit "`In [ ]:`" beginnen, Python-Code eingeben. Wenn Sie **SHIFT+Enter** drücken, wird der Code ausgeführt und die nächste Zelle wird ausgewählt. Führen Sie als Test den folgenden Code aus:

In [1]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



In [2]:
print("Hallo jupyter notebook")

Hallo jupyter notebook


- Dabei ist es möglich mehrere Zeilen zu programmieren, die dann nacheinander ausgeführt werden.

In [3]:
a = 5
b = 13
print(a + b)

18


- Falls die **letzte Zeile** in einer Code-Zelle `n` einen Wert beinhaltet, wird dieser als "`Out [n]:`" ausgegeben. Hierfür ist kein Aufruf von `print` notwendig.

In [4]:
print("Variablen bleiben über Zellen hinweg erhalten:")
a + b

Variablen bleiben über Zellen hinweg erhalten:


18

---

# Aufgabe 1 (Gleitpunktzahlen und -arithmetik)
**(a) Vollziehen Sie nach, dass die Gleitpunktaddition und -multiplikation (die Addition und Multiplikation im Computer, z. B. mit Python) nicht assoziativ sind. Dabei entspricht die Notation `1e-16` der Zahl $1\cdot 10^{-16}$ bzw. `xe-16` der Zahl $x\cdot 10^{-16}$.**

In [5]:
1 + 1e-16 + 1e-16

1.0

In [14]:
1.11e-16 + 1

1.0

In [15]:
(1+0.6e-8) * (1-0.6e-8) * (1-0.6e-8)

0.999999994

In [16]:
(1-0.6e-8) * (1-0.6e-8) * (1+0.6e-8)

0.9999999939999998

**(b) Die Definition der Maschinengenauigkeit zu gegebener Basis $B$ und Mantissenlänge $L_m$ der Vorlesung ist $\operatorname{eps}=\frac{B^{(1-L_m)}}{2}$. Im Allgemeinen ist $1+2\operatorname{eps}$ die kleinste Gleitpunktzahl größer als $1$. Im IEEE-Standard des Double-Precision Formats ($B=2, L_m=52$) ist $1+\operatorname{eps}$ diese kleinste Zahl (vgl. Bemerkung 1.3) im Skript):**

In [17]:
eps = 2**(-52)
print(eps)

2.220446049250313e-16


In [18]:
1 + 0.50001*eps == 1 + eps # Aufrunden

True

In [19]:
1 + 0.49999*eps == 1       # Abrunden

True

In [20]:
1 + eps == 0.8e-16 + 0.8e-16 + 1

True

In [21]:
# Es gibt viel kleinere Gleitpunktzahlen als die Maschinengenauigkeit
print(1e-308)
print(1e-323) # Hier wird die Normalisierung aufgehoben, um noch kleinere Zahlen darstellen zu können.
print(1e-324) # Exponentenunterlauf
print(1e309)  # Exponentenüberlauf

1e-308
1e-323
0.0
inf


**(c) Wir betrachten die Verteilung der normalisierten Gleitpunktzahlen $\mathrm{FL}$.**

In [22]:
import itertools
import matplotlib.pyplot as plt

B = 2 # Besser nur B = 2 oder B = 3
Lm = 3
emin = -1
Le = 2

plt.figure(figsize = (25,4))

# Erzeugung aller möglichen Koeffizienten-Tuple (a_1,...,a_{L_m}) durch itertools.product()
for al in itertools.product(range(B), repeat=Lm): 
    if al[0] != 0: # Normalisierung
        # Berechnung des Werts der Mantisse
        m = sum(al[l]*B**-(l+1) for l in range(Lm))
        
        # Zu jeder Mantisse werden alle möglichen Exponenten generiert
        for cl in itertools.product(range(B), repeat=Le):
            # Berechnung des Exponenten
            e = emin + sum(cl[l]*B**l for l in range(Le))
            
            # Plot der aktuellen Gleitpunktzahl auf einem Zahlenstrahl
            plt.plot(+m*B**e, 0, "+", color="k", markersize=10, markeredgewidth=2) 
            plt.plot(-m*B**e, 0, "+", color="k", markersize=10, markeredgewidth=2) 

# Markierung der Null
plt.plot(0, 0, "|", color="k", markersize=20, markeredgewidth=2)            
plt.show()    

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
print("Alle möglichen Mantissen:")
for al in itertools.product(range(B), repeat=Lm):
    if al[0] != 0: # Normalisierung
        m = sum(al[l]*B**-(l+1) for l in range(Lm))
        print("Symbolische Darstellung:", "".join(map(str, al)), "  Numerischer Wert", m)  

In [ ]:
print("Alle möglichen Exponenten:")
for cl in itertools.product(range(B), repeat=Le):
    e = emin + sum(cl[l]*B**l for l in range(Le))
    print("Symbolische Darstellung:", "".join(map(str, cl)), "  Numerischer Wert", e)          

**(d) Wir berechnen $e_{max}$, $\max \text{FL}$, $\min \text{FL}$, $\min \text{FL}_+$ $\max \text{Fl}_-$, $\text{eps}$**

In [ ]:
B = 2
Lm = 3
emin = -1
Le = 2

emax = emin + B ** Le - 1
max_FL = B ** emax * (1 - B ** - Lm)
min_FL = -max_FL
min_FL_p = B ** (emin - 1)
max_Fl_n = -min_FL_p
eps = B ** (1 - Lm) / 2
print(" emin     = {:2}\n emax     = {:2}\n max_FL   = {:6.3f}\n min_FL   = {:6.3f}\n min_FL_p = {:6.3f}\n max_Fl_n = {:6.3f}\n eps      = {:6.3f}".format(
    emin, emax, max_FL, min_FL, min_FL_p, max_Fl_n, eps))

In [ ]:
# IEEE Standard
B = 2
Lm = 52
emin = -1022
Le = 11

emax = 1023  # Formel liefert 1025, 1023 wird aber tatsächlich in IEEE Standard verwendet
max_FL = B ** emax * (1 - B ** - Lm)
min_FL = -max_FL
min_FL_p = B ** (emin - 1)
max_Fl_n = -min_FL_p
eps = (B ** (1 - Lm) / 2)
print(" emin     = {:5}\n emax     = {:5}\n max_FL   = {:14.3e}\n min_FL   = {:14.3e}\n min_FL_p = {:14.3e}\n max_Fl_n = {:14.3e}\n eps      = {:14.3e}".format(
    emin, emax, max_FL, min_FL, min_FL_p, max_Fl_n, eps))

**(d) Wir betrachten die im Übungsblatt definierten Funktionen**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def f(x):
    if x == 0:
        return 1
    else:
        return (np.exp(x)-1)/x
    
def g(x):
    y = np.exp(x)
    if y == 1:
        return 1
    else:
        return (y-1)/np.log(y)

x = np.linspace(10**(-308), 10**(-15), 100)
fx = [f(x_) for x_ in x]
gx = [g(x_) for x_ in x]

plt.plot(x, fx)
plt.plot(x, gx)

---

# Aufgabe 2 (Approximation von $\pi$, Auslöschung)

**(a) Berechnung einer Approximation durch eine stabile Rekursion**


In [ ]:
# Das Modul numpy stellt hier die Wurzelfunktion sqrt() und den Gleitpunktwert pi der Kreiszahl bereit.
# Der Zugriff auf die Wurzelfunktion erfolgt über np.sqrt(), den Gleitpunktwert von pi erhält man durch np.pi
import numpy as np

# Initialisierung der Rekursion
n = 6     # Anzahl der Dreiecke
g = 1     # bei 6 Dreiecken (gleichseitig) gilt g = r, also hier g = 1
pi_approx = 3
pi_approx_old = 0

while np.abs(pi_approx_old-pi_approx)>1e-15:
    n = 2*n                                # Verdopplung der Anzahl an Dreiecken
    g = g / np.sqrt(2 + np.sqrt(4 - g**2)) # Berechnung der neuen Grundseite auf stabile Weise: keine Auslöschung!!!
    pi_approx_old = pi_approx              # Speicherung der vorherigen Approximation für das STOPP-Kriterium
    pi_approx = n*g/2                      # Approximation von pi als Umfang/2
    # Formatierte Ausgabe
    print("pi_n: {:16.15f},   Fehler: {:20.16e}".format(pi_approx, np.pi - pi_approx))

**(b) Das Phänomen der Auslöschung bei der Subtraktion zweier Gleitpunktzahlen $x,y\in\mathrm{FL}$ mit $x\approx y$.**

In [ ]:
delta = 1e-9                  # Störung bzw. Rundungsfehler aus vorherigen Rechnungen
x = 0.1234567831234
y = 0.1234567832147 + delta   # Die Störung wirken sich in y nur auf hintere weniger signifikante Stellen der Mantisse aus 
x - y                         # das Ergebnis x - y wird maßgeblich von der Störung bestimmt!

**(c) Wir betrachten die quadratische Funktion $f(x)=(x-c)(x-c^{-1})=x^2-(c+c^{-1})x+1$ für $c\gg 1$. Die Nullstellen sind offenbar $x=c$ und $x=c^{-1}$. Wir berechnen die Nullstellen mit Hilfe der $pq$-Formel:**

In [ ]:
c = 20000000
x1 = (c + 1/c + np.sqrt((c + 1/c)**2-4))/2
x2 = (c + 1/c - np.sqrt((c + 1/c)**2-4))/2
print("Relativer Fehler bei der Berechnung der Nullstelle 1/c durch pq-Formel      = {}".format(np.abs(x2-1/c)*c))

x2 = 1/x1 # Berechnung der zweiten Nullstelle über Satz von Vieta
print("Relativer Fehler bei der Berechnung der Nullstelle 1/c durch Satz von Vieta = {}".format(np.abs(x2-1/c)*c))

Erklären Sie den großen relativen Fehler bei Verwendung der pq-Formel.

**(d) Erklären Sie, wo bei der Berechnung einer Approximation von $\pi$ Auslöschung auftritt:**

In [ ]:
# Initialisierung der Rekursion
n = 6     # Anzahl der Dreiecke
g = 1     # bei 6 Dreiecken (gleichseitig) gilt g = r, also hier g = 1

for k in range(30):
    n = 2*n
    
    # erste Variante ((1.1) im Skript)
    g = np.sqrt(2 - np.sqrt(4 - g**2))
    
    # zweite Variante ((1.2) im Skript)
    #g = g/np.sqrt(2+np.sqrt(4-g**2))
    pi_approx = n*g/2
    
print(pi_approx)